# 🤖 Introduction au RAG (Retrieval-Augmented Generation)

## Objectifs pédagogiques

Dans ce notebook, vous allez découvrir :
- 🔤 **Les embeddings** : comment transformer du texte en vecteurs
- 📏 **La similarité cosinus** : mesurer la proximité entre textes
- 📄 **Le chunking** : découper des documents longs
- 🔍 **La recherche vectorielle** avec FAISS
- 💬 **Le RAG** : combiner recherche et génération de texte

## 📦 Installation des dépendances

In [ ]:
# Installation des packages nécessaires
!pip install -q transformers torch faiss-cpu langchain langchain-community sentence-transformers langgraph langchain-huggingface

In [ ]:
# Imports nécessaires
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel, pipeline
from sentence_transformers import SentenceTransformer
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

print("✅ Toutes les dépendances sont installées !")

## 🔤 Partie 1 : Comprendre les Embeddings

Les embeddings transforment du texte en vecteurs numériques qui capturent le sens du texte.

In [ ]:
# CONFIGURATION - Modifiez ces paramètres !
# Essayez différents modèles d'embedding
EMBEDDING_MODEL_CONFIG = {
    # Modèle actuel (modifiez pour tester)
    "model_name": "sentence-transformers/all-MiniLM-L6-v2",  # Petit et rapide (384 dimensions)
    # Autres options à essayer :
    # "sentence-transformers/all-mpnet-base-v2"  # Plus précis (768 dimensions)
    # "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"  # Multilingue (384 dimensions)
}

# Chargement du modèle
print(f"🔄 Chargement du modèle : {EMBEDDING_MODEL_CONFIG['model_name']}")
embedder = SentenceTransformer(EMBEDDING_MODEL_CONFIG['model_name'])
print(f"✅ Modèle chargé ! Dimension des embeddings : {embedder.get_sentence_embedding_dimension()}")

In [ ]:
# Exemples de phrases à encoder
phrases_test = [
    "Le chat dort sur le canapé",
    "Le félin se repose sur le sofa",
    "La voiture roule sur l'autoroute",
    "Python est un langage de programmation"
]

# Génération des embeddings
embeddings = embedder.encode(phrases_test)

print("📊 Analyse des embeddings :")
for i, phrase in enumerate(phrases_test):
    print(f"\nPhrase {i+1}: '{phrase}'")
    print(f"  - Dimension de l'embedding : {embeddings[i].shape}")
    print(f"  - Premiers éléments : {embeddings[i][:5].round(3)}...")

### 🤔 Question 1 : Analyse des dimensions

**Répondez dans la cellule ci-dessous :**
- Quelle est la dimension des vecteurs d'embedding ?
- Pourquoi pensez-vous que des modèles différents ont des dimensions différentes ?
- Quel pourrait être l'impact de la dimension sur les performances ?

**Votre réponse :**

[Écrivez votre réponse ici]

## 📏 Partie 2 : Similarité Cosinus

La similarité cosinus mesure à quel point deux vecteurs pointent dans la même direction (de -1 à 1).

In [ ]:
# Calcul de la matrice de similarité
similarity_matrix = cosine_similarity(embeddings)

# Affichage des résultats
print("📊 Matrice de similarité cosinus :\n")
print("     ", end="")
for i in range(len(phrases_test)):
    print(f"  P{i+1}  ", end="")
print()

for i in range(len(phrases_test)):
    print(f"P{i+1}: ", end="")
    for j in range(len(phrases_test)):
        sim = similarity_matrix[i][j]
        print(f"{sim:.3f} ", end="")
    print()

# Interprétation
print("\n🔍 Interprétation des similarités :")
for i in range(len(phrases_test)):
    for j in range(i+1, len(phrases_test)):
        sim = similarity_matrix[i][j]
        print(f"\n'{phrases_test[i]}' <-> '{phrases_test[j]}'")
        print(f"  Similarité : {sim:.3f} - ", end="")
        if sim > 0.8:
            print("✅ Très similaire")
        elif sim > 0.5:
            print("🔶 Moyennement similaire")
        else:
            print("❌ Peu similaire")

In [ ]:
# EXERCICE : Testez vos propres phrases !
# Modifiez ces phrases pour explorer la similarité
MES_PHRASES = [
    "Le machine learning est fascinant",
    "L'apprentissage automatique est passionnant",
    "J'aime manger des pizzas",
    "L'intelligence artificielle révolutionne le monde"
]

# Calcul des embeddings et similarités
mes_embeddings = embedder.encode(MES_PHRASES)
mes_similarities = cosine_similarity(mes_embeddings)

# Affichage
print("🎯 Vos phrases et leurs similarités :\n")
for i in range(len(MES_PHRASES)):
    for j in range(i+1, len(MES_PHRASES)):
        sim = mes_similarities[i][j]
        print(f"'{MES_PHRASES[i]}' <-> '{MES_PHRASES[j]}'")
        print(f"  → Similarité : {sim:.3f}\n")

### 🤔 Question 2 : Analyse de la similarité

**Répondez dans la cellule ci-dessous :**
- Quelles paires de phrases ont la plus haute similarité ? Pourquoi ?
- La similarité capture-t-elle bien le sens sémantique ?
- Que se passe-t-il avec des synonymes vs des mots différents ?

**Votre réponse :**

[Écrivez votre réponse ici]

## 📄 Partie 3 : Chunking de Documents

Les modèles ont des limites de contexte. Il faut découper les documents longs en morceaux (chunks).

In [ ]:
# Documents sources pour notre base de connaissances
DOCUMENTS = [
    {
        "titre": "Introduction à Python",
        "contenu": """Python est un langage de programmation interprété, de haut niveau et à usage général. 
        Créé par Guido van Rossum et publié pour la première fois en 1991, Python a une philosophie de conception 
        qui met l'accent sur la lisibilité du code, notamment en utilisant des espaces blancs significatifs. 
        Il fournit des constructions qui permettent une programmation claire à petite et grande échelle. 
        Python dispose d'un système de type dynamique et d'une gestion automatique de la mémoire. 
        Il prend en charge plusieurs paradigmes de programmation, notamment la programmation procédurale, 
        orientée objet et fonctionnelle. Python dispose d'une bibliothèque standard complète et variée."""
    },
    {
        "titre": "Le Machine Learning",
        "contenu": """Le Machine Learning est une branche de l'intelligence artificielle qui permet aux systèmes 
        d'apprendre et de s'améliorer automatiquement à partir de l'expérience sans être explicitement programmés. 
        Le ML se concentre sur le développement de programmes informatiques qui peuvent accéder aux données 
        et les utiliser pour apprendre par eux-mêmes. Le processus d'apprentissage commence par des observations 
        ou des données, afin de rechercher des modèles dans les données et de prendre de meilleures décisions 
        à l'avenir. L'objectif principal est de permettre aux ordinateurs d'apprendre automatiquement sans 
        intervention humaine et d'ajuster leurs actions en conséquence."""
    },
    {
        "titre": "Les Réseaux de Neurones",
        "contenu": """Les réseaux de neurones artificiels sont des systèmes informatiques inspirés des réseaux 
        de neurones biologiques qui constituent le cerveau animal. Ces réseaux sont basés sur une collection 
        d'unités connectées appelées neurones artificiels, qui modélisent vaguement les neurones biologiques. 
        Chaque connexion peut transmettre un signal d'un neurone à un autre. Un neurone artificiel qui reçoit 
        un signal peut le traiter et signaler ensuite les neurones qui lui sont connectés. Dans les implémentations 
        courantes, le signal est un nombre réel et la sortie de chaque neurone est calculée par une fonction 
        non linéaire de la somme de ses entrées."""
    }
]

In [ ]:
# CONFIGURATION DU CHUNKING - Modifiez ces paramètres !
CHUNKING_CONFIG = {
    "chunk_size": 200,  # Taille des chunks en caractères (essayez 100, 200, 500)
    "chunk_overlap": 50,  # Chevauchement entre chunks (essayez 0, 50, 100)
    "separators": ["\n\n", "\n", ".", ",", " "]  # Séparateurs pour découper
}

# Création du text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNKING_CONFIG["chunk_size"],
    chunk_overlap=CHUNKING_CONFIG["chunk_overlap"],
    separators=CHUNKING_CONFIG["separators"]
)

# Application du chunking
print(f"📄 Configuration du chunking :")
print(f"  - Taille des chunks : {CHUNKING_CONFIG['chunk_size']} caractères")
print(f"  - Chevauchement : {CHUNKING_CONFIG['chunk_overlap']} caractères\n")

all_chunks = []
for doc in DOCUMENTS:
    chunks = text_splitter.split_text(doc["contenu"])
    print(f"\n📑 Document : {doc['titre']}")
    print(f"  - Longueur originale : {len(doc['contenu'])} caractères")
    print(f"  - Nombre de chunks : {len(chunks)}")
    
    for i, chunk in enumerate(chunks):
        all_chunks.append({
            "source": doc["titre"],
            "chunk_id": i,
            "content": chunk
        })
        print(f"\n  Chunk {i+1} ({len(chunk)} caractères) :")
        print(f"  '{chunk[:80]}...'")

### 🤔 Question 3 : Stratégies de chunking

**Expérimentez en modifiant les paramètres puis répondez :**
- Que se passe-t-il quand vous augmentez/diminuez la taille des chunks ?
- Quel est l'effet du chevauchement (overlap) ?
- Quels sont les avantages/inconvénients de chunks petits vs grands ?

**Votre réponse :**

[Écrivez votre réponse ici]

## 🔍 Partie 4 : Recherche Vectorielle avec FAISS

In [ ]:
# Création de la base vectorielle FAISS
print("🔨 Construction de la base vectorielle FAISS...")

# Création des embeddings pour tous les chunks
texts = [chunk["content"] for chunk in all_chunks]
metadatas = [{"source": chunk["source"], "chunk_id": chunk["chunk_id"]} for chunk in all_chunks]

# Initialisation du modèle d'embeddings pour Langchain
embeddings_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_CONFIG["model_name"]
)

# Création du vectorstore FAISS
vectorstore = FAISS.from_texts(
    texts=texts,
    embedding=embeddings_model,
    metadatas=metadatas
)

print(f"✅ Base vectorielle créée avec {len(texts)} chunks !")

In [ ]:
# CONFIGURATION DE LA RECHERCHE - Modifiez ces paramètres !
SEARCH_CONFIG = {
    "k": 3,  # Nombre de résultats à retourner (essayez 1, 3, 5)
    "search_type": "similarity",  # Type de recherche
}

# Questions de test
QUESTIONS_TEST = [
    "Qu'est-ce que Python ?",
    "Comment fonctionne l'apprentissage automatique ?",
    "Qu'est-ce qu'un neurone artificiel ?",
    "Qui a créé Python ?",
]

# Test de recherche
print(f"🔍 Test de recherche (top-{SEARCH_CONFIG['k']} résultats)\n")

for question in QUESTIONS_TEST:
    print(f"\n❓ Question : '{question}'")
    print("="*60)
    
    # Recherche
    results = vectorstore.similarity_search_with_score(
        question, 
        k=SEARCH_CONFIG["k"]
    )
    
    # Affichage des résultats
    for i, (doc, score) in enumerate(results):
        print(f"\n📄 Résultat {i+1} (score: {score:.3f})")
        print(f"   Source : {doc.metadata['source']} - Chunk {doc.metadata['chunk_id']}")
        print(f"   Contenu : '{doc.page_content[:100]}...'")

### 🤔 Question 4 : Analyse de la recherche

**Testez différentes valeurs de k puis répondez :**
- Les résultats retournés sont-ils pertinents ?
- Comment le nombre de résultats (k) affecte-t-il la qualité ?
- Que se passe-t-il si la question est ambiguë ?

**Votre réponse :**

[Écrivez votre réponse ici]

## 🤖 Partie 5 : Construction du système RAG complet

In [ ]:
# Chargement du modèle de génération
print("🤖 Chargement du modèle de chat...")

# CONFIGURATION DU MODÈLE - Modifiez ce paramètre !
GENERATION_MODEL = "microsoft/DialoGPT-small"  # Modèle léger pour Colab
# Autres options possibles :
# "google/flan-t5-small"  # Modèle T5 pour Q&A
# "facebook/blenderbot-400M-distill"  # Modèle de dialogue

# Chargement avec pipeline
generator = pipeline(
    "text-generation",
    model=GENERATION_MODEL,
    max_length=200,
    temperature=0.7,
    pad_token_id=50256
)

print(f"✅ Modèle {GENERATION_MODEL} chargé !")

In [ ]:
# Fonction RAG simple
def rag_query(question, k=3, verbose=True):
    """
    Fonction RAG complète :
    1. Recherche les documents pertinents
    2. Construit un contexte
    3. Génère une réponse
    """
    if verbose:
        print(f"\n🔍 Recherche pour : '{question}'")
    
    # Étape 1 : Recherche
    docs = vectorstore.similarity_search(question, k=k)
    
    # Étape 2 : Construction du contexte
    context = "\n\n".join([doc.page_content for doc in docs])
    
    if verbose:
        print(f"\n📚 Contexte trouvé ({len(docs)} documents) :")
        for i, doc in enumerate(docs):
            print(f"  - Doc {i+1}: {doc.metadata['source']}")
    
    # Étape 3 : Génération
    prompt = f"""Contexte : {context}
    
Question : {question}
    
Réponse basée sur le contexte : """
    
    response = generator(prompt, max_length=300, do_sample=True)[0]['generated_text']
    
    # Extraction de la réponse
    answer = response.split("Réponse basée sur le contexte : ")[-1].strip()
    
    return {
        "question": question,
        "context": context,
        "answer": answer,
        "sources": [doc.metadata for doc in docs]
    }

In [ ]:
# TESTEZ LE SYSTÈME RAG !
# Modifiez ces questions pour explorer
MES_QUESTIONS_RAG = [
    "Qu'est-ce que Python et qui l'a créé ?",
    "Comment fonctionne un réseau de neurones ?",
    "Quelle est la différence entre ML et les réseaux de neurones ?",
]

# Configuration
K_DOCUMENTS = 2  # Nombre de documents à récupérer (modifiez !)

# Test du RAG
for question in MES_QUESTIONS_RAG:
    print("\n" + "="*80)
    result = rag_query(question, k=K_DOCUMENTS, verbose=True)
    
    print(f"\n💬 Réponse générée :")
    print(f"{result['answer']}")
    
    print(f"\n📌 Sources utilisées :")
    for source in result['sources']:
        print(f"  - {source['source']} (chunk {source['chunk_id']})")

## 🎯 Partie 6 : Utilisation de LangGraph pour orchestrer le RAG

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Dict

# Définition de l'état
class RAGState(TypedDict):
    question: str
    context: str
    documents: List[Dict]
    answer: str
    k: int

# Fonctions pour chaque nœud
def retrieve_documents(state: RAGState) -> RAGState:
    """Récupère les documents pertinents"""
    print("📚 Étape 1: Recherche de documents...")
    docs = vectorstore.similarity_search(state["question"], k=state["k"])
    
    state["documents"] = [
        {"content": doc.page_content, "metadata": doc.metadata} 
        for doc in docs
    ]
    state["context"] = "\n\n".join([doc.page_content for doc in docs])
    
    print(f"  → {len(docs)} documents trouvés")
    return state

def generate_answer(state: RAGState) -> RAGState:
    """Génère la réponse basée sur le contexte"""
    print("🤖 Étape 2: Génération de la réponse...")
    
    prompt = f"""Contexte : {state['context']}
    
Question : {state['question']}
    
Réponse : """
    
    # Simulation de génération (remplacez par votre modèle)
    # Pour simplifier, on utilise une réponse basique
    if "Python" in state["question"]:
        state["answer"] = "D'après le contexte, Python est un langage de programmation créé par Guido van Rossum en 1991."
    elif "réseau" in state["question"] or "neurone" in state["question"]:
        state["answer"] = "Les réseaux de neurones sont des systèmes inspirés du cerveau, composés de neurones artificiels connectés."
    else:
        state["answer"] = "D'après les documents trouvés, " + state["context"][:150] + "..."
    
    print("  → Réponse générée")
    return state

# Construction du graphe
workflow = StateGraph(RAGState)

# Ajout des nœuds
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("generate", generate_answer)

# Définition du flux
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

# Compilation
rag_app = workflow.compile()

print("✅ Graphe LangGraph créé !")

In [ ]:
# Test du système RAG avec LangGraph
print("🚀 Test du RAG avec LangGraph\n")

# CONFIGURATION - Modifiez ces paramètres !
test_queries = [
    {"question": "Explique-moi Python", "k": 2},
    {"question": "Comment apprennent les machines ?", "k": 3},
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"❓ Question : {query['question']}")
    print(f"📊 Paramètres : k={query['k']}")
    print(f"{'='*60}\n")
    
    # Exécution du workflow
    result = rag_app.invoke(query)
    
    print(f"\n💬 Réponse finale : {result['answer']}")
    print(f"\n📌 Documents utilisés :")
    for doc in result['documents']:
        print(f"  - {doc['metadata']['source']}")

### 🤔 Question 5 : Analyse du système RAG complet

**Après avoir testé le système, répondez :**
- Quels sont les avantages d'utiliser un système RAG vs un LLM seul ?
- Comment le nombre de documents récupérés (k) affecte-t-il la réponse ?
- Quelles améliorations pourriez-vous suggérer ?

**Votre réponse :**

[Écrivez votre réponse ici]

## 📊 Expérience finale : Comparaison des configurations

Testez différentes configurations pour comprendre leur impact.

In [ ]:
# EXPÉRIENCE : Modifiez ces configurations !
EXPERIMENTS = [
    {
        "name": "Config 1 : Chunks petits, peu de docs",
        "chunk_size": 100,
        "chunk_overlap": 20,
        "k_documents": 1
    },
    {
        "name": "Config 2 : Chunks moyens, plus de docs",
        "chunk_size": 300,
        "chunk_overlap": 50,
        "k_documents": 3
    },
    {
        "name": "Config 3 : Grands chunks, beaucoup d'overlap",
        "chunk_size": 500,
        "chunk_overlap": 150,
        "k_documents": 2
    }
]

TEST_QUESTION = "Comment Python gère-t-il la mémoire ?"

print(f"🧪 Test de la question : '{TEST_QUESTION}'\n")

for config in EXPERIMENTS:
    print(f"\n{'='*60}")
    print(f"🔧 {config['name']}")
    print(f"  - Chunk size: {config['chunk_size']}")
    print(f"  - Overlap: {config['chunk_overlap']}")
    print(f"  - K documents: {config['k_documents']}")
    
    # Note : Dans un vrai test, vous recréeriez le vectorstore avec ces paramètres
    # Ici, on simule juste l'effet
    print(f"\n  → Impact attendu :")
    if config['chunk_size'] < 200:
        print("    • Contexte plus précis mais potentiellement incomplet")
    else:
        print("    • Contexte plus complet mais potentiellement moins focalisé")
    
    if config['k_documents'] > 2:
        print("    • Plus d'information mais risque de bruit")
    else:
        print("    • Information focalisée mais peut manquer des détails")

## 🎓 Récapitulatif et points clés

### Ce que vous avez appris :

1. **Embeddings** :
   - Transformation de texte en vecteurs numériques
   - Différents modèles = différentes dimensions
   - Capture du sens sémantique

2. **Similarité cosinus** :
   - Mesure de proximité sémantique (0 à 1)
   - Fonctionne bien avec les synonymes
   - Base de la recherche vectorielle

3. **Chunking** :
   - Nécessaire pour les limites de contexte
   - Trade-off : précision vs complétude
   - L'overlap aide à maintenir le contexte

4. **FAISS** :
   - Recherche efficace dans l'espace vectoriel
   - Permet de retrouver rapidement les documents pertinents

5. **RAG** :
   - Combine recherche et génération
   - Plus précis qu'un LLM seul
   - Permet d'utiliser des connaissances spécifiques

### 💡 Conseils pour optimiser un système RAG :

- **Qualité des embeddings** : Testez différents modèles
- **Stratégie de chunking** : Adaptez à votre contenu
- **Nombre de documents (k)** : Équilibre précision/bruit
- **Prompt engineering** : Guidez bien le modèle de génération
- **Métadonnées** : Utilisez-les pour filtrer/classer

### 🚀 Pour aller plus loin :

- Essayez d'autres modèles d'embedding (multilingues, domaine-spécifique)
- Explorez des stratégies de chunking avancées (sémantique, par paragraphe)
- Testez différents algorithmes FAISS (IVF, HNSW)
- Implémentez du re-ranking des résultats
- Ajoutez de la mémoire conversationnelle